In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import text
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/ipba-14-nlp-with-disaster-tweets/train.csv')
test=pd.read_csv('/kaggle/input/ipba-14-nlp-with-disaster-tweets/test.csv')
train.head()

In [ ]:
train.describe()

In [ ]:
text_data=train['text'].astype('str')
text_data= text_data.apply(lambda x: x.lower())
text_data.head()

In [ ]:
# function to remove special characters
text_data= text_data.apply(lambda x: re.sub("[^0-9a-zA-Z]",' ', x))
text_data.head()

In [ ]:
# removing stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
stop_words

def remove_sw(review):
    tokens = word_tokenize(review)
    tokens = [w for w in tokens if not w in stop_words]
    return " ".join(tokens)
text_data = text_data.apply(remove_sw)
text_data.head()

In [ ]:
# Stemming - Not required as text is for product description

ps = PorterStemmer()

def stem(text):
    return ' '.join([ps.stem(word) for word in text.split()])

text_data= text_data.apply(stem)
text_data.head()


In [ ]:
# initialise TF-IDF
#text_data=text_data.tolist()
#text_data.head()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 2))
matrix_tfidf = tf.fit_transform(text_data)

In [ ]:
matrix_tfidf.shape

In [ ]:
train['text_treated']=text_data
train.head()

In [ ]:
X=train['text_treated']
Y=train['target']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.25)
X_train.head()

In [ ]:

#X_train_tf=text.TfidfVectorizer(input=X_train.tolist(),ngram_range=(1,1))
#X_train_tf= X_train_tf.fit_transform(X_train.tolist())

#X_val_tf=text.TfidfVectorizer(input=X_val.tolist(),ngram_range=(1,1))
#X_val_tf= X_val_tf.fit_transform(X_val.tolist())

In [ ]:
tf=text.TfidfVectorizer()
X_train_tf=tf.fit_transform(X_train.tolist())

#X_val_tf=text.TfidfVectorizer()
X_val_tf= tf.transform(X_val.tolist())

In [ ]:
X_train_tf.shape

In [ ]:
X_val_tf.shape

In [ ]:
y_val.shape

In [ ]:
y_train.shape

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train_tf, y_train)

In [ ]:
mnb.score(X_val_tf, y_val)

In [ ]:
# Prediction on test set

test.head()

In [ ]:
test_id=test['id']

In [ ]:
test_text=test['text'].astype('str')
test_text= test_text.apply(lambda x: x.lower())
test_text.head()

In [ ]:
# function to remove special characters
test_text= test_text.apply(lambda x: re.sub("[^0-9a-zA-Z]",' ', x))
test_text.head()

In [ ]:
test_text = test_text.apply(remove_sw)
test_text.head()

In [ ]:
test_text= test_text.apply(stem)
test_text.head()

In [ ]:
test_text=test_text.tolist()

test_matrix_tfidf=tf.transform(test_text)

In [ ]:
predictions=mnb.predict(test_matrix_tfidf)

In [ ]:
predictions = pd.Series(predictions)
ids = pd.Series(test_id)

pred_df = pd.concat([ids, predictions], keys = ['id', 'target'], axis = 1)

pred_df.to_csv('submission.csv',index = False)